# Transiting Exoplanets Workbook

This notebook demonstrates how to download Kepler photometry and use it to detect a transiting planet.

# Installation

Run the following command to install the `lightkurve` package

In [ ]:
!pip install lightkurve

# Import Necessary Packages

In [ ]:
import lightkurve as lk
import matplotlib.pylab as plt
%matplotlib inline

# Search for photometry for a particular Kepler system

Enter the star name (the example uses Kepler-10). This cell will output a table of all the datasetes of Kepler data that exist for this star. For Kepler-10, there are a lot of datasets, so we probably will not use all of them (to keep the computation fast). Note that a star can have multiple datasets per quarter (a single dataset does not span the entire quarter).

In [113]:
search_result = lk.search_lightcurve('Kepler-10', mission='Kepler', exptime=60)
search_result

#,mission,year,author,exptime,target_name,distance
,,,,s,,arcsec
0,Kepler Quarter 02,2009,Kepler,60,kplr011904151,0.0
1,Kepler Quarter 03,2009,Kepler,60,kplr011904151,0.0
2,Kepler Quarter 03,2009,Kepler,60,kplr011904151,0.0
3,Kepler Quarter 03,2009,Kepler,60,kplr011904151,0.0
4,Kepler Quarter 04,2010,Kepler,60,kplr011904151,0.0
5,Kepler Quarter 05,2010,Kepler,60,kplr011904151,0.0
6,Kepler Quarter 05,2010,Kepler,60,kplr011904151,0.0
7,Kepler Quarter 05,2010,Kepler,60,kplr011904151,0.0
8,Kepler Quarter 06,2010,Kepler,60,kplr011904151,0.0


# Download Kepler Photometry

The following lines will download some Kepler Photometry. Note that we will only download the first 8 entries to keep the data processing times manageable. Note that if you look at a different star, you may need to change how many entries to keep: more entries means more data which means better sensitivity, while other stars may not have 8 entries of data so you may need to choose less.


In [ ]:
lc_collection = search_result[:8].download_all() # only download the first 8 entries
lc_collection

In [ ]:

# Create a larger figure for clarity
fig, ax = plt.subplots(figsize=(20,5))
# Plot the light curve collection
lc_collection.plot(ax=ax);

# Process the lightcurve

To process the lightcurve, we will: 

 1. Stitch all the data together
 2. Use a filter to remove variations in the light curve due to instrumental and stellar effects (this scale is chosen to be much larger than the transit duration so any transits are not filtered out)
 3. Extreme outliters are removed

In [ ]:
# Flatten the light curve
lc = lc_collection.stitch().flatten(window_length=901).remove_outliers(sigma=6)

lc.plot();

# Search For Planets Using a Periodogram Search

This is a plot of periodogram power versus orbital period. Strong peaks indicate there is a periodic signal here that matches a transit pattern.

The periodogram search can be finncky if not all instrumental/stellar effects are removed. To mitigate that, pick a search period range (the `period` variable) that is close to the period you expect (e.g., from previous detections). Astronomers trying to detect new planets don't have such a luxury!

In [ ]:
import numpy as np
# Create array of periods to search
period = np.linspace(0.7, 0.9, 1000) # units of days
# Plot the periodogram 
bls = lc.to_periodogram(method='bls', period=period, frequency_factor=500);
bls.plot();

## If you don't see a peak in the periodogram above corresponding to your period:

 1. Change the search period
 2. Try including more data
 3. Pick a different, easier planet to detect

# Measured planet parameters and Phase-folded light curve

This analysis returns the measured planet period (`planet_period`), planet transit duration (`planet_duration`), and the start time of one of the transits (`planet_t0`). 

With the measured period, we can phase-fold the light curve to stack all the data points together. However, there is a lot of noise still, so it may be hard to see the transit.

In [ ]:
planet_period = bls.period_at_max_power
planet_t0 = bls.transit_time_at_max_power
planet_duration = bls.duration_at_max_power

print(planet_period, planet_t0, planet_duration)


folded_lc = lc.fold(period=planet_period, epoch_time=planet_t0)
folded_lc.errorbar();
plt.xlim([-0.1, 0.1])

Plot wiht transit Model. 

To average out the noise, we will bin data points close in phase together. To additionally help guide the eye, we will plot a transit model using our best fit parameters. 

In [ ]:
# Create a BLS model using the BLS parameters
planet_model = bls.get_transit_model(period=planet_period,
                                     transit_time=planet_t0,
                                     duration=planet_duration)

ax = lc.fold(planet_period, planet_t0).bin(.0005).scatter()
planet_model.fold(planet_period, planet_t0).plot(ax=ax, c='r', lw=2)
ax.set_xlim(-0.5, 0.5);




# Plot individual transits

This plot below visualizes how many transits there are in the data. There amy be too many to tell. 

In [ ]:
ax = lc.scatter();
planet_model.plot(ax=ax, c='r', label='Planet Transit Model');

# Compute the transit depth.

Rather than estimate it from the data, which is noisy, we instead use the model trasit to measure the transit depth (which is just the maximium change in flux). 

In [ ]:
transit_depth = (np.max(planet_model.flux) - np.min(planet_model.flux))
print(transit_depth)